In [14]:
from typing import get_args

from bcs import BCSz

from api_dev.types import AI, DIO, Motor

ai = get_args(AI.__value__)
motor = get_args(Motor.__value__)
dio = get_args(DIO.__value__)

In [15]:
from pydantic import BaseModel, Field
from pydantic_settings import BaseSettings, SettingsConfigDict


class Connection(BaseSettings):
    addr: str = Field(alias="BCS_SERVER_ADDRESS")
    port: int = Field(alias="BCS_SERVER_PORT")
    model_config = SettingsConfigDict(env_file=".env", extra="ignore")


# Expected Data Structures for AI returns and for motor position returns


class AiData(BaseModel):
    chan: str
    period: float
    time: float
    data: list[float]


class AiResponse(BaseModel):
    success: bool
    error_description: str = Field(alias="error description")
    log: bool = Field(alias="log?")
    not_found: list[str] = []
    chans: list[AiData]
    API_delta_t: float

    model_config = {"populate_by_name": True}


class MotorData(BaseModel):
    motor: str
    position: float
    position_raw: float
    goal: float
    goal_raw: float
    status: int
    time: float


class MotorResponse(BaseModel):
    success: bool
    error_description: str = Field(alias="error description")
    log: bool = Field(alias="log?")
    not_found: list[str] = []
    data: list[MotorData]
    API_delta_t: float

    model_config = {"populate_by_name": True}

    def __repr__(self):
        return f"<MotorResponse success={self.success} data={self.data}>"


class DioData(BaseModel):
    enabled: bool
    data: bool


class DioResponse(BaseModel):
    success: bool
    error_description: str = Field(alias="error description")
    log: bool = Field(alias="log?")
    chans: list[str]
    not_found: list[str] = []
    enabled: list[bool]
    data: list[bool]
    API_delta_t: float

    model_config = {"populate_by_name": True}


config = Connection()
config

Connection(addr='localhost', port=5577)

In [31]:
import asyncio
from dataclasses import dataclass
from typing import Generic, TypeVar

import numpy as np
import pandas as pd

T = TypeVar("T", bound=AI | Motor | DIO)


@dataclass
class TabularResponse:
    state: pd.DataFrame
    status: pd.DataFrame


class RsoxsAccessor(Generic[T]):
    """Class structure responsible for accessing AI's and Motors."""

    def __init__(
        self, server: BCSz.BCSServer, kind: type[T], *, readonly: bool = False
    ) -> None:
        self.server = server
        self.kind = kind
        self.readonly = readonly

    async def __getitem__(
        self, key: T | tuple[T, ...]
    ) -> AiResponse | MotorResponse | DioResponse:
        # Handle both single key and multiple keys
        if isinstance(key, str):
            keys = (key,)
        else:
            keys = key

        # Check if all keys are valid for this accessor type
        if any(k not in get_args(self.kind.__value__) for k in keys):
            raise KeyError(f"{keys} is not a valid {self.kind}")

        if self.kind is AI:
            result = await self.server.get_acquired_array(chans=list(keys))
            return AiResponse(**result)
        elif self.kind is Motor:
            result = await self.server.get_motor(motors=list(keys))
            return MotorResponse(**result)
        elif self.kind is DIO:
            result = await self.server.get_di(chans=list(keys))
            return DioResponse(**result)
        else:
            raise NotImplementedError(
                f"Accessor for type {self.kind} is not implemented."
            )

    async def set(self, key: str, value):
        if self.readonly:
            raise PermissionError("This accessor is read-only.")

        # Check if key is valid for this accessor type
        if key not in get_args(self.kind.__value__):
            raise KeyError(f"{key} is not a valid {self.kind}")

        if self.kind is Motor:
            await self.server.command_motor(
                commands=["Backlash Move"], motors=[key], goals=[value]
            )
        elif self.kind is DIO:
            if not isinstance(value, (int, bool)):
                raise ValueError("DIO value must be an integer or boolean.")
            await self.server.set_do(chan=key, value=bool(value))
        else:
            raise NotImplementedError(
                f"Setting value for type {self.kind} is not implemented."
            )

    async def table(self, keys: list[str]) -> TabularResponse:
        """
        Retrieve data for multiple keys and return it in a tabular format.

        Parameters
        ----------
        keys : list[str]
            List of keys to retrieve data for.

        Returns
        -------
        TabularResponse
            DataFrame containing the requested data in tabular format.

        Examples
        --------
        >>> response = await motor.df(['chan1', 'chan2'])
        >>> print(response.state)
            chan1   chan2
        0   0.1     0.2
        >>> print(response.status)
            motor  position   goal   status   time
        0   motor1  10.0      15.0   0        1625247600.0
        1   motor2  20.0      25.0   0        1625247600.0
        """
        import pandas as pd

        records = []
        state = {}
        for key in keys:
            response = await self.__getitem__(key)
            if self.kind is AI:
                for chan_data in response.chans:
                    records.append(chan_data.model_dump())
                    state[chan_data.chan] = chan_data.data
            elif self.kind is Motor:
                for motor_data in response.data:
                    state[motor_data.motor] = [motor_data.position]
                    records.append(
                        {
                            "motor": motor_data.motor,
                            "position": motor_data.position,
                            "goal": motor_data.goal,
                            "status": motor_data.status,
                            "time": motor_data.time,
                        }
                    )
            elif self.kind is DIO:
                for chan, en, da in zip(
                    response.chans, response.enabled, response.data
                ):
                    state[chan] = da
                    records.append({"chan": chan, "enabled": en, "data": da})
        return TabularResponse(
            state=pd.DataFrame(state),
            status=pd.DataFrame(records),
        )

    async def from_df(self, df: pd.DataFrame, map: callable) -> None:
        """Set multiple values from a DataFrame."""
        if self.readonly:
            raise PermissionError("This accessor is read-only.")

        for key in df.columns:
            if key not in get_args(self.kind.__value__):
                raise KeyError(f"{key} is not a valid {self.kind}")

        for index, row in df.iterrows():
            for key in df.columns:
                await self.set(key, row[key])


class RsoxsServer(BCSz.BCSServer):
    CONFIG = Connection()

    def __init__(self):
        super().__init__()

        self.ccd_ready: bool = False

        self.ai = RsoxsAccessor[AI](self, AI, readonly=True)
        self.motor = RsoxsAccessor[Motor](self, Motor)
        self.dio = RsoxsAccessor[DIO](self, DIO)
        #  Some default metadata values

    async def connect_with_env(self):
        """Connect to the server using environment variables"""
        import io
        from contextlib import redirect_stdout

        buff = io.StringIO()
        with redirect_stdout(buff):
            await self.connect(**self.CONFIG.model_dump())
            self.__public_key = buff.getvalue().split(" ")[-1]

    @classmethod
    async def create(cls) -> "RsoxsServer":
        """Factory method to create and connect the server"""
        instance = cls()
        await instance.connect_with_env()
        return instance

    # -------- Instrument Setup Methods --------#
    async def setup_ccd(self) -> None:
        """Setup the CCD camera for data acquisition."""
        # Check if the cached property is allready set
        if self.ccd_ready:
            return
        res = await self.get_instrument_driver_status(name="CCD")
        if not res["success"]:
            raise RuntimeError(f"Failed to get CCD status: {res['error description']}")
        self.ccd_ready = res["running"]

    async def _set_ccd_temp(self) -> None:
        """Set the CCD temperature."""
        #  Assumes the ccd is setup already and set the temperature
        if not self.ccd_ready:
            raise RuntimeError("CCD is not setup. Don't run this function alone")
        current_temp = (await self.get_state_variable("Camera Temp"))["numeric_value"]
        set_temp = (await self.get_state_variable("Camera Temp Setpoint"))[
            "numeric_value"
        ]
        if set_temp > -40:
            print("Camera temperature setpoint too high, setting to -40C")
            set_temp = -40
            await self.set_state_variable("Camera Temp Setpoint", -40)

        # Wait for camera to reach target temperature
        tolerance = 1.0  # Temperature tolerance in degrees C
        max_wait_time = 300  # Maximum wait time in seconds
        check_interval = 2  # Check every 2 seconds

        elapsed_time = 0
        while abs(current_temp - set_temp) > tolerance:
            if elapsed_time >= max_wait_time:
                raise TimeoutError(
                    f"Camera temperature did not stabilize within {max_wait_time}s. "
                    f"Current: {current_temp}C, Target: {set_temp}C"
                )

            await asyncio.sleep(check_interval)
            elapsed_time += check_interval
            current_temp = (await self.get_state_variable("Camera Temp"))[
                "numeric_value"
            ]
            print(
                f"Waiting for camera to cool... Current: {current_temp:.1f}C, Target: {set_temp}C"
            )

        print(f"Camera temperature stabilized at {current_temp:.1f}C")

    def snap(self, exposure: float) -> np.ndarray:
        """Take a snapshot with the specified exposure time."""

        #  Ensure the camera is setup


server = await RsoxsServer.create()

In [34]:
result_1 = await server.ai[["Photodiode", "TEY signal"]]
result_1

AiResponse(success=True, error_description='no error', log=False, not_found=[], chans=[AiData(chan='Photodiode', period=0.0005, time=3845136223.89423, data=[-0.0526489144281018, -0.0507421567279429, -0.049755902746295, -0.0518599112414375, -0.0514982847810717, -0.0521229123035951, -0.0517612858431444, -0.0520571620380498, -0.0515311599138275, -0.0518927863742037, -0.0517941609759078, -0.0509722826571053, -0.0512681588518074, -0.0538324192092246, -0.0479148953157566, -0.0527804149592694, -0.0521229123035951, -0.0508079069934135, -0.0516297853121004, -0.0510051577898464, -0.051925661506971, -0.0508079069934135, -0.0522215377019205, -0.0512024085863117, -0.0522544128346976, -0.0509394075243652, -0.051728410710382, -0.051925661506971, -0.05166266044486, -0.0533392922169196, -0.052550289029737, -0.0525174138969508, -0.0521229123035951, -0.051925661506971, -0.0526489144281018, -0.052977665756052, -0.0518599112414375, -0.0521229123035951, -0.051925661506971, -0.052977665756052, -0.05084078212

In [23]:
import pandas as pd
import uncertainties as un

std = np.std(result_1.chans[0].data)
mean = np.mean(result_1.chans[0].data)
err = std / (len(result_1.chans[0].data) ** 0.5)

series = pd.Series([un.ufloat(mean, err)], name="Photodiode")
series

0    -0.05185+/-0.00006
Name: Photodiode, dtype: object

In [49]:
from bcs.BCSz import MotorStatus


def actuate(
    prefix="", shutter="Shutter Output", count_time_s=1.0, delay_after_move_s=0.2
):
    def decorator(func):
        async def wrapper(*args, **kwargs):
            server = kwargs.get("server")

            result = await func(*args, **kwargs)
            await asyncio.sleep(delay_after_move_s)
            await server.dio.set(shutter, True)

            try:
                await asyncio.sleep(count_time_s)
                return result
            finally:
                # Close shutter
                await server.dio.set(shutter, False)

        return wrapper

    return decorator


def safely(prefix="", timeout: float = 60.0):
    def decorator(func):
        async def wrapper(*args, **kwargs):
            server = kwargs.get("server")
            try:
                result = await func(*args, **kwargs)
                motor = kwargs.get("motor")
                if motor is not None:
                    while not await move_complete(server, motor):
                        await asyncio.sleep(0.001)
                return result
            except Exception as e:
                print(
                    f"{prefix}: Error occurred - {e}"
                    if prefix
                    else f"Error occurred - {e}"
                )
                raise

        return wrapper

    return decorator


async def move_complete(server, motor_status: list):
    motor_info = (await server.get_motor(motors=list(motor)))["data"]
    for m in motor_info:
        if not MotorStatus(m["status"]).is_set(MotorStatus.MOVE_COMPLETE):
            return False
    return True


@actuate(
    prefix="Move X", shutter="Light Output", count_time_s=1.0, delay_after_move_s=0.2
)
@safely()
async def move_x(server):
    return await server.motor.set("Sample X", 10.0)


await move_x(server=server)

In [37]:
await server.motor.set("Sample X", 15)
await server.get_motor(motors=list(motor))

{'success': True,
 'error description': 'no error',
 'log?': False,
 'not found': [],
 'data': [{'motor': 'Sample Azimuthal Rotation',
   'position': -0.0,
   'position_raw': 0.0,
   'goal': 0.0,
   'goal_raw': 0.0,
   'status': 65592,
   'time': 3845052645.63627},
  {'motor': 'Piezo Vertical',
   'position': 0.0,
   'position_raw': 0.0,
   'goal': 0.0,
   'goal_raw': 0.0,
   'status': 66592,
   'time': 0.0},
  {'motor': 'Piezo Horiz',
   'position': 0.0,
   'position_raw': 0.0,
   'goal': 0.0,
   'goal_raw': 0.0,
   'status': 66592,
   'time': 0.0},
  {'motor': 'Sample X',
   'position': 30.0,
   'position_raw': -60000.0,
   'goal': 30.0,
   'goal_raw': -60000.0,
   'status': 24,
   'time': 3845052645.63727},
  {'motor': 'Sample Y',
   'position': 12.6875,
   'position_raw': 25375.0,
   'goal': 12.6875,
   'goal_raw': 25375.0,
   'status': 65568,
   'time': 3845052645.63827},
  {'motor': 'Sample Z',
   'position': 0.672,
   'position_raw': -1344.0,
   'goal': 0.672,
   'goal_raw': -13

Motor Sample Azimuthal Rotation is in position -0.0
Motor Piezo Vertical is in position 0.0
Motor Piezo Horiz is in position 0.0
Motor Sample X is in position 15.0
Motor Sample Y is in position 12.6875
Motor Sample Z is in position 0.672
Motor Sample Theta is in position 90.0
Motor Sample Y Scaled is in position 11.9223501262212
Motor CCD Theta is in position 0.0
Motor Beam Stop is in position -3.05
Motor Pollux CCD X is in position 0.0
Motor Pollux CCD Y is in position 0.0
Motor CCD X is in position 6.0
Motor CCD Y is in position 100.0
Motor T-2T is in position 90.0
Motor Beamline Energy is in position 408.232850471794
Motor Mono 101 Grating is in position -33.6094713099964
Motor Beamline Energy Goal is in position 0.0
Motor Entrance Slit width is in position 35.0078125
Motor Exit Slit Top is in position 5061.0546875
Motor Exit Slit Bottom is in position 2312.20703125
Motor Exit Slit Left is in position 4708.0078125
Motor Exit Slit Right is in position 5410.0
Motor Horizontal Exit Sli

In [7]:
await server.motor.set("Sample X", 10)

In [9]:
response = await server.ai.table(["AI 3 Izero", "Photodiode"])
response.state

,AI 3 Izero,Photodiode
0,-0.022842,170.573351
1,-0.017582,171.822608
2,-0.024157,171.428106
3,-0.011665,171.921234
4,-0.022842,173.630744
...,...,...
1995,-0.035006,172.578738
1996,-0.017911,173.499243
1997,-0.017911,172.677363
1998,-0.016925,172.184235
